## Load Plot Data from a Local FIA Database

In [1]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

import os
import sqlite3

import numpy as np
import pandas as pd

try:
  import pysiteindex as pysi
except:
  print('pysiteindex is not installed, site index calculations will not be available')
  pysi = None

try:
  import pyfvs
  import pyfvs.keywords as kwds
except:
  print('pyfvs is not installed, FVS features will not be available')
  pyfvs = None

try:
  import pynvel
except:
  print('pynvel is not installed, Volume calculation will not be available')
  pynvel = None

In [2]:
db_path = 'c:/data/fia/fiadb_or.db'

In [68]:
class FIADB(object):
  plot_sql = """
    with c as (
      select plt_cn
        , avg(stdage * condprop_unadj)/sum(condprop_unadj) as stdage
        , avg(stdage * condprop_unadj)/sum(condprop_unadj) as stdage
      from cond
      group by plt_cn
      )

    select p.cn as plot_cn, p.plot, p.prev_plt_cn, p.invyr
      , p.measyear as msmt_year, p.measmon as msmt_month, p.measday as msmt_day
      , p.elev as elevation
      , g.fvs_variant, g.fvs_region, g.fvs_forest, g.fvs_loc_cd as fvs_location
      , g.lat, g.lon
      , c.stdage
      , case when not p0.measyear is null then p.measyear-p0.measyear else 0 end as meas_len
      
    from plot as p
      left join plotgeom as g
        on g.cn=p.cn
      
      left join c
        on c.plt_cn=p.cn

      left join plot as p0
		    on p0.cn=p.prev_plt_cn
    """

  subplot_sql = """
    select cn as subplot_cn, plt_cn as plot_cn, plot, prev_sbp_cn as prev_subplot_cn
      , subp, subp_status_cd, invyr
      , subpcond
      , slope, aspect

    from subplot
    """

  tree_sql = """
    select t.cn as tree_cn, t.tree, t.prev_tre_cn as prev_tree_cn
      , t.plt_cn as plot_cn, t.plot, t.subp, t.invyr
      --, azimuth, dist
      , t.statuscd, t.spcd, t.tpa_unadj
      , t.dia
      --, case when t.statuscd=1 then t.dia-t.prevdia else 0 end as dg
      , t.prevdia
      , t.ht
      , t.actualht
      , t.boleht
      , t.sawht
      , t.formcl
      , t.totage, t.bhage
      , t.cr
      , t.cull
      , t.damloc1, t.damtyp1, t.damsev1
      , t.damloc2, t.damtyp2, t.damsev2
      , t.volcfgrs, t.volcfnet, t.volcsgrs, t.volcsnet

      , t.inc5yr_pnwrs as t.inc5yr
      , t.inc10yr_pnwrs as t.inc10yr

      , g.fvs_variant
      , 'tree' as source

    from tree as t
      inner join plotgeom as g
        on g.cn=t.plt_cn
    """

  seedling_sql = """
    select s.cn as seedling_cn, s.plt_cn as plot_cn, s.plot, s.subp, s.invyr
      , s.spcd, s.tpa_unadj
      , g.fvs_variant
      , 'seedling' as source

    from seedling as s
      inner join plotgeom as g
        on g.cn=s.plt_cn
    """

  site_tree_sql = """
    select s.cn as site_tree_cn, s.tree, s.plt_cn as plot_cn, s.plot, s.prev_sit_cn as prev_site_cn
      , s.invyr
      , s.spcd, s.dia, s.ht, s.agedia
      , s.sitree, s.sibase, s.sibase_fvs, s.sieqn_ref_cd_fvs, s.method
      --, azimuth, dist

      , g.fvs_variant

    from sitetree as s
      inner join plotgeom as g
        on g.cn=s.plt_cn
    """

  def __init__(self, db_path, scale='PLOT'
      , plot_filter=None, subplot_filter=None
      , tree_filter=None, seedling_filter=None
      , site_tree_filter=None
      ):
    self.db_path = db_path
    self.scale = scale.upper() # PLOT, COND, SUBPLOT
    self.plot_filter = plot_filter
    self.subplot_filter = subplot_filter
    self.tree_filter = tree_filter
    self.seedling_filter = seedling_filter
    self.site_tree_filter = site_tree_filter

    self._conds = None
    self._plots = None
    self._subplots = None
    self._trees = None
    self._seedlings = None
    self._site_trees = None

    if not os.path.exists(self.db_path):
      raise ValueError(f'FIA database does not exist: {self.db_path}')

    if self.scale!='PLOT':
      raise NotImplementedError('Query scales other than PLOT are not yet implemented')

  @property
  def plots(self):
    if self._plots is None:
      self.fetch_plots()

    return self._plots

  def fetch_plots(self):
    """Fetch plot records from a local FIA SQLite database"""
    print('Loading plots')

    if self.plot_filter:
      sql = f'select * from ({self.plot_sql}) as f where ({self.plot_filter})'
    
    else:
      sql=self.plot_sql

    with sqlite3.connect(db_path) as conn:
      self._plots = pd.read_sql(sql, conn)

    print(f'Loaded {self._plots.shape[0]} plot records')

  @property
  def subplots(self):
    if self._subplots is None:
      self.fetch_subplots()

    return self._subplots

  def fetch_subplots(self):
    """Fetch subplot records for the current selection of plots"""
    if self._plots is None:
      self.fetch_plots()
    
    print('Loading subplots')

    cns = ','.join([f"'{cn}'" for cn in self.plots['plot_cn'].unique()])
    sql = f'select * from ({self.subplot_sql}) as f where plot_cn in ({cns})'

    if not self.subplot_filter is None:
      sql += f' and ({self.subplot_filter})'

    with sqlite3.connect(db_path) as conn:
      self._subplots = pd.read_sql(sql, conn)

    print(f'Loaded {self._subplots.shape[0]} subplot records')

  ## TODO: Append results to self._plots
  def average_subplot_topo(self):
    """Compute aggregate topographic values for the current selection of subplots"""
    self.subplots['slope'] = np.rad2deg(np.arctan(self.subplots['slope']*0.01))
    # Average sine and cosine of aspect
    self.subplots['aspect_sin'] = np.sin(np.deg2rad(self.subplots['aspect']))
    self.subplots['aspect_cos'] = np.cos(np.deg2rad(self.subplots['aspect']))

    gb = self.subplots.groupby('plot_cn', as_index=False)

    df = gb[['slope','aspect_sin','aspect_cos']].agg('mean')
    df['aspect'] = np.rad2deg(np.arctan2(df['aspect_sin'], df['aspect_cos']))
    df['aspect'] += 360 * (df['aspect']<0)

    # Convert slope back to percent
    df['slope'] = np.tan(np.deg2rad(df['slope']))*100

    return df

  ## TODO: Apend results to self._trees
  def apply_fvs_spp(self, trees, inplace=True):
    """Set the fvs_species for a treelist"""
    if not inplace:
      trees = trees.copy()
    
    trees['fvs_species'] = ''

    variants = trees['fvs_variant'].unique()
    for var in variants:
      f = pyfvs.fvs.FVS(var)
      spp_codes = {v:k for k,v in f.spp_fia_codes.items()}
      m = trees['fvs_variant']==var
      trees.loc[m,'fvs_species'] = trees.loc[m,'spcd'].replace(spp_codes)

    if not inplace:
      return trees
    else:
      return None

  @property
  def trees(self):
    if self._trees is None:
      self.fetch_trees()

    return self._trees

  def fetch_trees(self):
    """Fetch tree records for the currently selected subplots"""
    if self._subplots is None:
      self.fetch_subplots()
    
    print('Loading sample trees')

    cns = ','.join([f"'{cn}'" for cn in self.subplots['plot_cn'].unique()])
    sql = f'select * from ({self.tree_sql}) as f where plot_cn in ({cns})'

    if not self.tree_filter is None:
      sql += f' and ({self.tree_filter})'

    with sqlite3.connect(db_path) as conn:
      self._trees = pd.read_sql(sql, conn)

    self._trees['spcd'] = self.trees['spcd'].astype(int)

    self.apply_fvs_spp(self._trees)

    # self.trees = self.trees.merge(self.plots.loc[:,['plot_cn','fvs_variant']], on='plot_cn', how='left')
    # self.trees['fvs_species'] = ''
    # dfs = []
    # for g,rows in self.trees.groupby()
    
    print(f'Loaded {self._trees.shape[0]} tree records')

  @property
  def site_trees(self):
    if self._site_trees is None:
      self.fetch_site_trees()
    return self._site_trees

  def fetch_site_trees(self):
    """Fetch site index sample records for the currently selected subplots"""
    if self._subplots is None:
      self.fetch_subplots()
    
    print('Loading site trees')

    cns = ','.join([f"'{cn}'" for cn in self.subplots['plot_cn'].unique()])
    sql = f'select * from ({self.site_tree_sql}) as f where plot_cn in ({cns})'

    if not self.site_tree_filter is None:
      sql += f' and ({self.site_tree_filter})'

    with sqlite3.connect(db_path) as conn:
      self._site_trees = pd.read_sql(sql, conn)

    self._site_trees['spcd'] = self.site_trees['spcd'].astype(int)

    self.apply_fvs_spp(self._site_trees)

    print(f'Loaded {self._site_trees.shape[0]} site tree records')
    
  @property
  def seedlings(self):
    if self._seedlings is None:
      self.fetch_seedlings()

    return self._seedlings

  def fetch_seedlings(self):
    """Fetch seedling records for the currently selected subplots"""
    if self._subplots is None:
      self.fetch_subplots()
    
    cns = ','.join([f"'{cn}'" for cn in self.subplots['plot_cn'].unique()])
    sql = f'select * from ({self.seedling_sql}) as f where plot_cn in ({cns})'

    if not self.seedling_filter is None:
      sql += f' and ({self.seedling_filter})'

    with sqlite3.connect(db_path) as conn:
      seedlings = pd.read_sql(sql, conn)

    seedlings['spcd'] = seedlings['spcd'].astype(int)

    # Add a sequential number to each seedling record by subplot
    def tree_num(rows):
      rows['tree'] = np.arange(1,rows.shape[0]+1)
      return rows

    seedlings['tree'] = 0
    self._seedlings = seedlings.groupby(['plot_cn','subp'], as_index=False).apply(tree_num)

    self.apply_fvs_spp(self._seedlings)

    print(f'Loaded {self._seedlings.shape[0]} seedling records')

  @property
  def plot_site_index(self):
    """Calculate average site index by plot and FVS species using sample site trees"""
    site_trees = self.calc_site_index()
    plot_si = site_trees.groupby(['plot_cn','fvs_species'], as_index=False)['fvs_site_index'].mean()
    return plot_si
    
  def calc_site_index(self, trees=None):
    """Estimate site index for data frame of site trees"""
    # Use pySiteIndex to estimate SI using FVS curves
    
    if trees is None:
      trees = self.site_trees

    trees['fvs_site_index'] = None
    
    def apply_site(rows):
      variant = rows.iloc[0]['fvs_variant']
      spp = rows.iloc[0]['fvs_species']
      sc = pysi.si.FVS_SiteCurve(variant,spp)
      rows['fvs_site_index'] = sc.v_site_index(rows.agedia, rows.ht)
      return rows

    tree_gb = self.site_trees.groupby(['fvs_variant','fvs_species'])
    
    dfs = []
    for g,rows in tree_gb:
      dfs.append(apply_site(rows))

    trees = pd.concat(dfs)

    return trees

  def gen_fvs(self):
    """Generate FVS stand and tree records for the selected data"""
    plot_si = self.plot_site_index
    plot_si.set_index('plot_cn', inplace=True)
    trees = self.trees.copy()
    trees.set_index('plot_cn', inplace=True)
    plot_topo = self.average_subplot_topo()
    plot_topo.set_index('plot_cn', inplace=True)

    # Convert FIA tree attributes to FVS attributes
    trees['plot_id'] = 1 #trees['subp']
    trees['tree_id'] = trees['tree']
    
    # Tree history/status
    # Default to live trees
    trees['history'] = 1
    # Long dead trees
    m = trees['statuscd']==2
    trees.loc[m,'history'] = 8
    # Recently cut
    m = trees['statuscd']==3
    trees.loc[m,'history'] = 6

    trees['prob'] = trees['tpa_unadj']
    trees['species'] = trees['spcd']
    trees['diameter'] = trees['dia']
    trees['diameter_growth'] = 0 #trees['prevdia']
    trees['height'] = trees['ht']
    trees['height_growth'] = 0.0
    m = (trees['actualht']>0) & (trees['actualht']<trees['ht'])
    trees['total_height'] = 0
    # trees.loc[m,'total_height'] = trees.loc[m,'actualht']
    trees['crown'] = trees['cr']
    trees['age'] = trees['bhage']
    
    for plot in self.plots.itertuples():
      plot_cn = getattr(plot,'plot_cn')
      topo = plot_topo.loc[plot_cn]
      kw = kwds.KeywordSet(title=f"CN-{plot_cn}")
      kw += kwds.STDINFO(
        location=getattr(plot,'fvs_location')
        , aspect=topo['aspect'].astype(int)
        , slope=topo['slope'].astype(int)
        , elevation=getattr(plot,'elevation')
        # , age=20
        )

      # kw += kwds.DESIGN(0,1,999) # Defaults to 1 acres plot, i.e. direct TPA expansion
      kw += kwds.DESIGN()
      kw += kwds.INVYEAR(getattr(plot, 'msmt_year'))
      # Growth measurement methods, growth years
      # 1=previous diameter measurement
      kw += kwds.GROWTH(1, getattr(plot, 'meas_len'))

      try:
        spp_si = plot_si.loc[[plot_cn,]]
        for row in spp_si.itertuples():
          kw += kwds.SITECODE(row.fvs_species, row.fvs_site_index)
      
      except KeyError:
        pass

      plot_trees = trees.loc[[plot_cn,]]

      # Yield the FVS data for the current plot
      yield {
        'cn':plot_cn
        ,'scale':self.scale
        ,'keywords':kw
        ,'trees':plot_trees
        }

## TODO: Make pre-fetching data optional
## TODO: 

fia = FIADB(db_path, plot_filter='fvs_location in (612)')

# plots = fia.plots
# subplots = fia.subplots

# plot_topo = fia.average_subplot_topo()

# trees = fia.trees
# seedlings = fia.seedlings
# site_trees = fia.site_trees

# seedlings.rename(columns={'seedling_cn':'tree_cn'}, inplace=True)
# all_trees = pd.concat([trees,seedlings], axis=0, ignore_index=False)

# fia.apply_fvs_spp(fia.trees)
# fia.apply_fvs_spp(fia.seedlings)
# fia.apply_fvs_spp(fia.site_trees)

# site = fia.calc_site_index(fia.site_trees)


In [69]:
# x['trees']

In [70]:

def grow_plot(variant,kw,trees,id):

  r = []
  for k in ('calib','nocalib'):
    fm = pyfvs.fvs.FVS(variant, workspace='c:/temp/fia_grow', cleanup=False)

    kw += kwds.NUMCYCLE(1)
    kw += kwds.TIMEINT(0,10)
    kw += kwds.NOTRIPLE()
    if k=='NOCALIB':
      kw += kwds.NOCALIB()
    
    fm.keywords = kw

    fm.inventory_trees = trees

    fm.run()

    sim_trees = fm.trees
    sim_trees['id'] = id
    sim_trees['grow_method'] = k

    r.append(dict(
      id=id
      ,calib=k
      ,summary=fm.summary
      ,trees=sim_trees
    ))

  return r

grown_trees = []
fvs_data = fia.gen_fvs()
for x in fvs_data:
# x = fvs_data.__next__()


  id = x['cn']
  r = grow_plot('PN',x['keywords'], x['trees'], id)
  grown_trees.append(pd.concat([o['trees'] for o in r]))

Loading plots
Loaded 1046 plot records
Loading subplots
Loaded 4184 subplot records
Loading site trees
Loaded 5158 site tree records
Loading sample trees
Loaded 31021 tree records


KeyError: "None of [Index(['26367669010900'], dtype='object', name='plot_cn')] are in the [index]"

In [57]:
grown_trees

[     cycle  year  plot_seq  tree_seq  tree_id  spp_seq  age  live_tpa  \
 0        0  2001         1         1      100       16   14  6.018046   
 1        0  2001         1         2      101       16   14  6.018046   
 2        0  2001         1         3      102       16   15  6.018046   
 3        0  2001         1         4      103       16   10  6.018046   
 4        0  2001         1         5      104       16   14  6.018046   
 ..     ...   ...       ...       ...      ...      ...  ...       ...   
 136      0     0         0         0        0        0    0  0.000000   
 137      0     0         0         0        0        0    0  0.000000   
 138      0     0         0         0        0        0    0  0.000000   
 139      0     0         0         0        0        0    0  0.000000   
 140      0     0         0         0        0        0    0  0.000000   
 
      live_dbh  dbh_incr  ...  cr_ratio  ht_merch_cuft  ht_merch_bdft  \
 0         6.6       0.0  ...      80

In [39]:
# x['trees']
# x['summary']

In [53]:
# r['trees'].groupby('year')[['live_tpa',]].sum()
# r['summary']
x = pd.concat([o['trees'] for o in r])
x

,cycle,year,plot_seq,tree_seq,tree_id,spp_seq,age,live_tpa,live_dbh,dbh_incr,...,cr_ratio,ht_merch_cuft,ht_merch_bdft,cuft_total,cuft_net,bdft_net,defect_cuft,defect_bdft,id,calib
0,0,2001,1,1,100,16,14,6.018046,6.6,0.0,...,80.0,0.0,0.0,3.4,0.0,0.0,0.0,0.0,41118132010497,calib
1,0,2001,1,2,101,16,14,6.018046,6.1,0.0,...,80.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,41118132010497,calib
2,0,2001,1,3,102,16,15,6.018046,6.4,0.0,...,80.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,41118132010497,calib
3,0,2001,1,4,103,16,10,6.018046,6.4,0.0,...,80.0,0.0,0.0,2.9,0.0,0.0,0.0,0.0,41118132010497,calib
4,0,2001,1,5,104,16,14,6.018046,6.8,0.0,...,85.0,0.0,0.0,2.9,0.0,0.0,0.0,0.0,41118132010497,calib
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,0,0,0,0,0,0,0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41118132010497,nocalib
137,0,0,0,0,0,0,0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41118132010497,nocalib
138,0,0,0,0,0,0,0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41118132010497,nocalib
139,0,0,0,0,0,0,0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41118132010497,nocalib


In [18]:
trees = r['trees']
m = trees['year']==2001
trees.loc[m,:]

,cycle,year,plot_seq,tree_seq,tree_id,spp_seq,age,live_tpa,live_dbh,dbh_incr,...,ht_incr,cr_width,cr_ratio,ht_merch_cuft,ht_merch_bdft,cuft_total,cuft_net,bdft_net,defect_cuft,defect_bdft
0,0,2001,1,1,100,16,14,6.018046,6.600000,0.0,...,0.0,13.264104,49.0,0.000000,0.000000,3.400000,0.000000,0.0,0.0,0.0
1,0,2001,1,2,101,16,14,6.018046,6.100000,0.0,...,0.0,12.313810,42.0,0.000000,0.000000,3.000000,0.000000,0.0,0.0,0.0
2,0,2001,1,3,102,16,15,6.018046,6.400000,0.0,...,0.0,12.879089,46.0,0.000000,0.000000,3.000000,0.000000,0.0,0.0,0.0
3,0,2001,1,4,103,16,10,6.018046,6.400000,0.0,...,0.0,12.880143,46.0,0.000000,0.000000,2.900000,0.000000,0.0,0.0,0.0
4,0,2001,1,5,104,16,14,6.018046,6.800000,0.0,...,0.0,13.546103,50.0,0.000000,0.000000,2.900000,0.000000,0.0,0.0,0.0
5,0,2001,1,6,105,16,11,6.018046,5.200000,0.0,...,0.0,10.822334,34.0,0.000000,0.000000,1.700000,0.000000,0.0,0.0,0.0
6,0,2001,1,7,106,16,11,6.018046,5.400000,0.0,...,0.0,11.172100,36.0,0.000000,0.000000,2.100000,0.000000,0.0,0.0,0.0
7,0,2001,1,8,109,16,10,74.965279,4.800000,0.0,...,0.0,9.884703,27.0,0.000000,0.000000,1.500000,0.000000,0.0,0.0,0.0
8,0,2001,1,9,112,16,11,6.018046,5.900000,0.0,...,0.0,11.787760,37.0,0.000000,0.000000,2.500000,0.000000,0.0,0.0,0.0
9,0,2001,1,10,107,16,11,6.018046,5.900000,0.0,...,0.0,11.849222,38.0,0.000000,0.000000,2.700000,0.000000,0.0,0.0,0.0


In [27]:
si = fia.plot_site_index
si.set_index('plot_cn', inplace=True)

# x = si.groupby(level=0)['fvs_species'].count()
# x[x>1]

# si.loc['23849369010900']
si.loc[['41118132010497',],:]

,fvs_species,fvs_site_index
plot_cn,,
41118132010497,WH,122.949219


In [70]:
# fia.site_trees.to_clipboard()
# fia.calc_site_index()

fvs_data = fia.gen_fvs()
x = fvs_data.__next__()

Loading plots
Loaded 1046 plot records
Loading subplots
Loaded 4184 subplot records
Loading site trees
Loaded 5158 site tree records
Loading sample trees
Loaded 31021 tree records


In [73]:
# print(x['keywords'])
display(x['trees'])

,tree_cn,tree,prev_tree_cn,plot,subp,invyr,statuscd,spcd,tpa_unadj,dia,...,damloc2,damtyp2,damsev2,volcfgrs,volcfnet,volcsgrs,volcsnet,fvs_variant,source,fvs_species
plot_cn,,,,,,,,,,,,,,,,,,,,,
41118132010497,42188249010497,100,None,59747,1,2001,1,202,6.018046,6.6,...,0.0,NaN,NaN,2.735951,2.735951,NaN,NaN,PN,tree,DF
41118132010497,42188250010497,101,None,59747,1,2001,1,202,6.018046,6.1,...,0.0,NaN,NaN,2.261935,2.261935,NaN,NaN,PN,tree,DF
41118132010497,42188251010497,102,None,59747,1,2001,1,202,6.018046,6.4,...,0.0,NaN,NaN,2.352563,2.352563,NaN,NaN,PN,tree,DF
41118132010497,42188252010497,103,None,59747,1,2001,1,202,6.018046,6.4,...,0.0,NaN,NaN,2.271111,2.271111,NaN,NaN,PN,tree,DF
41118132010497,42188253010497,104,None,59747,1,2001,1,202,6.018046,6.8,...,0.0,NaN,NaN,2.194447,2.194447,NaN,NaN,PN,tree,DF
41118132010497,42188254010497,105,None,59747,1,2001,1,202,6.018046,5.2,...,0.0,NaN,NaN,0.925070,0.925070,NaN,NaN,PN,tree,DF
41118132010497,42188255010497,106,None,59747,1,2001,1,202,6.018046,5.4,...,0.0,NaN,NaN,1.264691,1.264691,NaN,NaN,PN,tree,DF
41118132010497,42188258010497,109,None,59747,1,2001,1,202,74.965282,4.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PN,tree,DF
41118132010497,42188260010497,112,None,59747,2,2001,1,202,6.018046,5.9,...,0.0,NaN,NaN,1.785016,1.785016,NaN,NaN,PN,tree,DF


In [140]:
fia.site_trees['fvs_site_index'] = None
    
def apply_site(rows):
  try:
    variant = rows.iloc[0]['fvs_variant']
    spp = rows.iloc[0]['fvs_species']
  except:
    print(rows)
    raise
  sc = pysi.si.FVS_SiteCurve(variant,spp)
  rows['fvs_site_index'] = sc.v_site_index(rows.agedia, rows.ht)
  return rows
  # si = sc.v_site_index(rows.agedia, rows.ht)
  # return pd.Series(dict(site_index=si))

tree_gb = fia.site_trees.groupby(['fvs_variant','fvs_species'])
trees = tree_gb.apply(apply_site)

plot_si = trees.groupby(['plot_cn','fvs_species'])['fvs_site_index'].mean()
plot_si.to_clipboard()

In [152]:
f = pyfvs.fvs.FVS('PN')
# f.fvslib.globals.kodfor = np.array(0, dtype=int)
f.fvslib.forkod()
f.fvslib.globals.ifor

array(2, dtype=int32)

In [142]:
trees.loc[trees['plot_cn']=='12893428010497']

,site_tree_cn,tree,plot_cn,plot,prev_site_cn,invyr,spcd,dia,ht,agedia,sitree,sibase,sibase_fvs,sieqn_ref_cd_fvs,method,fvs_variant,fvs_species,fvs_site_index
45,12893433010497,1,12893428010497,52979,None,2005,202,36.1,174,72,141,50,NaN,None,2,PN,DF,140.527344
46,12893436010497,2,12893428010497,52979,None,2005,202,55.9,210,132,131,50,NaN,None,2,PN,DF,130.639648
47,512474049126144,3,12893428010497,52979,None,2005,351,7.3,42,17,74,50,20.0,ALRU01,1,PN,RA,-inf
48,512474050126144,4,12893428010497,52979,None,2005,351,8.7,55,19,90,50,20.0,ALRU01,1,PN,RA,56.884766
49,512474051126144,5,12893428010497,52979,None,2005,351,9.5,53,17,93,50,20.0,ALRU01,1,PN,RA,58.935547


In [60]:
trees.loc[trees['CN']=='374382093489998']

,CN,PLT_CN,PREV_TRE_CN,INVYR,STATECD,UNITCD,COUNTYCD,PLOT,SUBP,TREE,...,CORE_LENGTH_PNWRS,CULTURALLY_KILLED_PNWRS,DIA_EST_PNWRS,GST_PNWRS,INC10YR_PNWRS,INC5YRHT_PNWRS,INC5YR_PNWRS,RING_COUNT_INNER_2INCHES_PNWRS,RING_COUNT_PNWRS,SNAG_DIS_CD_PNWRS
480183,374382093489998,290008527489998,324921899489998,2015,41,1,41,89939,1,101,...,NaN,NaN,44.0,None,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
cns = ','.join([f"'{cn}'" for cn in fia.subplots['plot_cn'].unique()])
sql = f'select * from ({fia.tree_sql}) as f where plot_cn in ({cns})'

with sqlite3.connect(db_path) as conn:
  trees = pd.read_sql(sql, conn)

# trees.loc[trees['cn']=='374382093489998']
trees

,tree_cn,tree,plot_cn,plot,subp,azimuth,dist,statuscd,spcd,tpa_unadj,...,damtyp1,damsev1,damloc2,damtyp2,damsev2,volcfgrs,volcfnet,volcsgrs,volcsnet,source
0,12802973010497,100,12802934010497,98752,1,2005,None,1,202.0,6.018046,...,NaN,NaN,NaN,NaN,NaN,40.363935,40.363935,39.323170,39.323170,tree
1,12802976010497,101,12802934010497,98752,1,2005,None,1,202.0,6.018046,...,NaN,NaN,NaN,NaN,NaN,183.938224,183.938224,182.646372,182.646372,tree
2,12802987010497,104,12802934010497,98752,3,2005,None,1,202.0,6.018046,...,NaN,NaN,NaN,NaN,NaN,162.985854,162.985854,161.836795,161.836795,tree
3,12802994010497,106,12802934010497,98752,4,2005,None,1,202.0,6.018046,...,NaN,NaN,NaN,NaN,NaN,71.537413,71.537413,70.745588,70.745588,tree
4,12802997010497,107,12802934010497,98752,4,2005,None,1,202.0,6.018046,...,NaN,NaN,NaN,NaN,NaN,115.442357,115.442357,114.584595,114.584595,tree
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31016,9255138010901,160,9255017010901,69019,4,2005,None,1,202.0,6.018046,...,NaN,NaN,NaN,NaN,NaN,74.976558,71.227730,74.187422,70.478051,tree
31017,9255139010901,161,9255017010901,69019,4,2005,None,1,202.0,0.999188,...,NaN,NaN,NaN,NaN,NaN,353.370117,353.370117,350.896121,350.896121,tree
31018,9255141010901,162,9255017010901,69019,4,2005,None,1,263.0,6.018046,...,NaN,NaN,NaN,NaN,NaN,110.448078,93.880866,109.651744,93.203982,tree
31019,9255142010901,163,9255017010901,69019,4,2005,None,1,202.0,6.018046,...,NaN,NaN,NaN,NaN,NaN,25.365281,25.365281,24.163929,24.163929,tree


In [22]:
sql = """select * from fvs_treeinit_plot"""
with sqlite3.connect(db_path) as conn:
  fvs_trees = pd.read_sql(sql, conn)

fvs_sum = fvs_trees.groupby(['plot_cn','plot_id'])['tpa_unadj'].sum()

,tree_cn,tree,plot_cn,plot,subp,azimuth,dist,statuscd,spcd,tpa_unadj,...,damsev1,damloc2,damtyp2,damsev2,volcfgrs,volcfnet,volcsgrs,volcsnet,source,invyr
0,12802973010497,100,12802934010497,98752,1,2005.0,None,1,202.0,6.018046,...,NaN,NaN,NaN,NaN,40.363935,40.363935,39.323170,39.323170,tree,NaN
1,12802976010497,101,12802934010497,98752,1,2005.0,None,1,202.0,6.018046,...,NaN,NaN,NaN,NaN,183.938224,183.938224,182.646372,182.646372,tree,NaN
2,12802987010497,104,12802934010497,98752,3,2005.0,None,1,202.0,6.018046,...,NaN,NaN,NaN,NaN,162.985854,162.985854,161.836795,161.836795,tree,NaN
3,12802994010497,106,12802934010497,98752,4,2005.0,None,1,202.0,6.018046,...,NaN,NaN,NaN,NaN,71.537413,71.537413,70.745588,70.745588,tree,NaN
4,12802997010497,107,12802934010497,98752,4,2005.0,None,1,202.0,6.018046,...,NaN,NaN,NaN,NaN,115.442357,115.442357,114.584595,114.584595,tree,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31471,8709892010901,1,8709873010901,96840,1,NaN,NaN,1,263.0,224.895847,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,seedling,2006.0
31472,8709893010901,1,8709873010901,96840,1,NaN,NaN,1,351.0,74.965282,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,seedling,2006.0
31473,8709894010901,1,8709873010901,96840,2,NaN,NaN,1,263.0,74.965282,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,seedling,2006.0
31474,8709895010901,1,8709873010901,96840,2,NaN,NaN,1,351.0,74.965282,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,seedling,2006.0


In [9]:
# sc = pysi.si.FVS_SiteCurve('PN', 'DF')
spx = pyfvs.fvs.FVS('PN').spp_fia_codes

In [15]:
trees.groupby('plot_cn')['tpa_unadj'].sum()

plot_cn
12802934010497     30.090230
12854856010497    423.491364
12860093010497    293.318644
12891256010497    157.399630
12892786010497    367.579408
                     ...    
8709873010901     123.358484
8720362010901     460.120226
8723694010901     128.377342
8723844010901     240.721840
9255017010901     186.444836
Name: tpa_unadj, Length: 854, dtype: float64

In [58]:
subplots.loc[subplots['plot_cn']=='12854856010497'] #['aspect'].mean()

,subplot_cn,plot_cn,plot,prev_subplot_cn,subp,subp_status_cd,invyr,subpcond,slope,aspect,slope_deg,aspect_sin,aspect_cos
4,12854870010497,12854856010497,85153,None,4,1,2005,1,10.0,360.0,5.710593,-2.449294e-16,1.000000
5,12854868010497,12854856010497,85153,None,3,1,2005,1,0.0,0.0,0.000000,0.000000e+00,1.000000
6,12854866010497,12854856010497,85153,None,2,1,2005,1,30.0,286.0,16.699244,-9.612617e-01,0.275637
7,12854864010497,12854856010497,85153,None,1,1,2005,1,15.0,260.0,8.530766,-9.848078e-01,-0.173648


In [46]:
df = fia.subplots
df['slope_degrees'] = np.rad2deg(np.arctan(df['slope']*0.01))
df

,subplot_cn,plot_cn,plot,prev_subplot_cn,subp,subp_status_cd,invyr,subpcond,slope,aspect,slope_degrees
0,12802953010497,12802934010497,98752,None,4,1,2005,1,10.0,271.0,5.710593
1,12802951010497,12802934010497,98752,None,3,1,2005,1,10.0,280.0,5.710593
2,12802947010497,12802934010497,98752,None,1,1,2005,1,10.0,300.0,5.710593
3,12802949010497,12802934010497,98752,None,2,1,2005,2,10.0,208.0,5.710593
4,12854870010497,12854856010497,85153,None,4,1,2005,1,10.0,360.0,5.710593
...,...,...,...,...,...,...,...,...,...,...,...
4179,15121118010497,9827912010902,50908,None,4,2,2010,1,NaN,NaN,NaN
4180,15121130010497,9827963010902,50875,None,1,2,2010,1,NaN,NaN,NaN
4181,15121131010497,9827963010902,50875,None,2,2,2010,1,NaN,NaN,NaN
4182,15121132010497,9827963010902,50875,None,3,2,2010,1,NaN,NaN,NaN
